# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

In [1]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-11-09 15:22:48--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   136MB/s    in 0.3s    

2024-11-09 15:22:49 (136 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

In [2]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

In [3]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

In [4]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

In [15]:
def map1(x):
    # Extrair o ID e a classificação (sentiment)
    # Retornar uma tupla apenas se for negativo
    if x.sentiment == 'neg':
        return (1, int(x.id))  # Key é 1 para todos para fazer a soma total
    return (1, 0)  # Retorna 0 para não afetar a soma

## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

In [16]:
def reduceByKey1(x, y):
    # Somar os valores
    return x + y

## Aplicação do map/reduce e visualização do resultado

In [17]:
resultado = imdb_df.rdd \
    .map(map1) \
    .reduceByKey(reduceByKey1) \
    .collect()

print(f"A soma dos IDs dos reviews negativos é: {resultado[0][1]}")

A soma dos IDs dos reviews negativos é: 459568555


# Questão 2:

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e uma tupla com a soma das palavras de cada texto como valor.

In [19]:
def count_words(text):
    # Remove pontuações e converte para minúsculas antes de contar
    if text is None:
        return 0
    words = text.strip().split()
    return len(words)

In [20]:
def map2(x):
    # Retorna uma tupla com o sentiment como chave e (palavras_pt, palavras_en) como valor
    pt_words = count_words(x.text_pt)
    en_words = count_words(x.text_en)
    return (x.sentiment, (pt_words, en_words))

## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment".

In [21]:
def reduceByKey2(x, y):
    # Soma as contagens de palavras mantendo português e inglês separados
    return (x[0] + y[0], x[1] + y[1])

## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [22]:
resultado = imdb_df.rdd \
    .map(map2) \
    .reduceByKey(reduceByKey2) \
    .collect()

for sentiment, (pt_count, en_count) in resultado:
    if sentiment == 'neg':
        diferenca = pt_count - en_count
        print(f"Total de palavras em português (negativos): {pt_count}")
        print(f"Total de palavras em inglês (negativos): {en_count}")
        print(f"Diferença (português - inglês): {diferenca}")


Total de palavras em português (negativos): 5455273
Total de palavras em inglês (negativos): 5400324
Diferença (português - inglês): 54949
